# Perform a looping with the quadrotor

# Goal of the project

The goal of this project is to control a 2D quadrotor to get it to perform acrobatic moves. The controller will be designed using an SQP solver.

Please submit your code as a runnable Jupyter Notebook.

## 2D quadrotor

The quadrotor is depicted in the following figure
<img src='quadrotor.png' width="300">


The quadrotor model is written as
$$\begin{align} 
\dot{p_x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{p_y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $p_x$ is the horizontal and $p_y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $x = [p_x, v_x, p_y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines the problem and provides all the useful information about the robot and methods to simulate and animate it as shown below.

You can access the different parameters of the model in the following way:

In [15]:
import quadrotor

print("Mass    =", quadrotor.MASS)
print("Length  =", quadrotor.LENGTH)
print("Inertia =", quadrotor.INERTIA)
print("Dt      =", quadrotor.DT)
print("state size   =", quadrotor.DIM_STATE)
print("control size =", quadrotor.DIM_CONTROL)

Mass    = 0.5
Length  = 0.15
Inertia = 0.1
Dt      = 0.04
state size   = 6
control size = 2




## Part 1 - Setting up the trajectory Optimization (50 points)
1. Discretize the system dynamics using the Euler method seen in class - write the time discretization step as $\Delta t$ (use symbols not numbers for the mass, etc)
2. We would like the quadrotor to perfom a looping. Find and implement a suitable cost function to perform a looping and add constraint to maintain the thrust of each rotor between $0$ and $10$. Solve the problem using your own implementation of a SQP (levarging your code from Homework 2) with a large horizon to check that you can do a looping.
3. Show plots of all the states and controls of the robot as a function of time. Describe your design choices (in a concise manner) in the report.

## Part 2 - Model predictive control (MPC) (50 points)
1. Use the trajectory optimization method from Part I to design a MPC controller and test it using the simulator below. In particular, verify that it can handle perturbations by calling the ```quadrotor.simulate``` function with ```disturbance = True``` (when setting disturbance to True, the simulator will generate a random perturbation every 1 second). Simulate your controller for 10 seconds, plot the state anc control evolution.
2. Explain your intended design in the report, including the cost function and found control law

The ```quadrotor.simulate``` function takes as an input an initial state, a controller, the number of discrete time steps and a boolean value to indicate the presence of perturbation. The controller has to be a function taking as an input a state and time index and outputting a control vector.

To visualize the trajectory, use the ```quadrotor.animate_robot``` function
and show the animation (show the plots in your report).

## Bonus (10 points)
Add a state constraint to perform the looping while maintening a positive altitude. Use the origin as an initial state.

In [16]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython
from scipy.linalg import block_diag

def controller(x, t):
    return np.zeros(2)

x_init = np.array([0, 0, 0., 0 ,0, 0])
horizon_length = 100
t, state, u = quadrotor.simulate(x_init, controller, horizon_length, disturbance=True)
quadrotor.animate_robot(state, u)

Your browser does not support the video tag.


## Solution : Part 1 - Setting up the trajectory Optimization (50 points)
### 1. Discretize the system dynamics using the Euler method seen in class - write the time discretization step as $\Delta t$ (use symbols not numbers for the mass, etc)

To discretise the system dynamics, we shall simply rearrange the given dynamic equations such that multiplication with $\Delta t$ will provide the necessary addition to the current state resulting in the next state. This is shown below - 

$$\begin{align*}
    \dot{p}_{x_n} &= v_{x_n} \\
    \dot{v}_{x_n} &= - \frac{(u_{1_n} + u_{2_n}) \sin \theta_n}{m} \\
    \dot{p}_{y_n} &= v_{y_n} \\
    \dot{p}_{y_n} &= \frac{(u_{1_n} + u_{2_n}) \cos \theta_n}{m} - g \\
    \dot{\theta}_n &= \omega_n \\
    \dot{\omega}_n &= r \frac{(u_{1_n} - u_{2_n})}{I} = \frac{L(u_{1_n} - u_{2_n})}{I}  \quad, \quad L = \text{ Length of quadrotor}
\end{align*}$$

This gives us the equality constraints, 
$$\begin{align*}
    p_{x_{n+1}} &= p_{x_n} + \Delta t v_{x_n} \\
    v_{x_{n+1}} &= v_{x_n} - \Delta t \frac{(u_{1_n} + u_{2_n}) \sin \theta_n}{m} \\
    p_{y_{n+1}} &= p_{y_n} + \Delta t v_{y_n} \\
    v_{y_{n+1}} &= v_{y_n} + \Delta t \left(\frac{(u_{1_n} + u_{2_n}) \cos \theta_n}{m} - g \right)\\
    \theta_{n+1} &= \theta_{n} + \Delta t \omega_n \\
    \omega_{n+1} &= \omega_{n} + \Delta t \frac{L(u_{1_n} - u_{2_n})}{I}
\end{align*}$$

As we can see, a couple of the constraints are non-linear. So we must first find a linear approximation of the constraints at a given guess $\bar{x}$ in a form $G(\bar{x}) \Delta x = g(\bar{x})$ where $\Delta x$ represents a small variation around $\bar{x}$. To do that, we shall perform the Taylor Expansion of the constraints around $\bar{x}$, which will be given by, 

$$\begin{align*}
    \Delta p_{x_{n+1}} &= \Delta p_{x_n} + \Delta t \Delta v_{x_n} \\
    \Delta v_{x_{n+1}} &= \Delta v_{x_n} - \Delta t \Delta \theta_n \frac{(u_{1_n} + u_{2_n}) \cos \theta_n}{m} - \Delta t (\Delta u_{1_n} +  \Delta u_{2_n} )\frac{\sin \theta_n}{m}\\
    \Delta p_{y_{n+1}} &= \Delta p_{y_n} + \Delta t \Delta v_{y_n} \\
    \Delta v_{y_{n+1}} &= \Delta v_{y_n} - \Delta t \Delta \theta_n \frac{(u_{1_n} + u_{2_n}) \sin \theta_n}{m} + \Delta t (\Delta u_{1_n} + \Delta u_{2_n}) \frac{\cos \theta_n}{m}\\
    \Delta \theta_{n+1} &= \Delta \theta_{n} + \Delta t \Delta \omega_n \\
    \Delta \omega_{n+1} &= \Delta \omega_{n} + \Delta t \frac{L(\Delta u_{1_n} - \Delta u_{2_n})}{I}
\end{align*}$$

Finally, we shall put it in the matrix form $Ay = b$ where $A$ would be the Jacobian of the Constraints, $\Delta y$ would be the small variation around the combined state and control vector and $b$ would be tne constraints themselves. 
$\begin{equation}\notag A(y) \Delta y = b(y) \end{equation}$
Where, 
$$
A = \begin{bmatrix} 
    1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\
    0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\
    0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\
    0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\
    0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\
    0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \cdots \\
    1 & \Delta t & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & \cdots \\
    0 & 1 & 0 & 0 & -\Delta t \frac{(u_{1_n} + u_{2_n}) \cos \theta_n}{m} & 0 & -\Delta t \frac{\sin \theta_n}{m} & -\Delta t \frac{\sin \theta_n}{m} & 0 & -1 & 0 & 0 & 0 & 0 & \cdots \\ 
    0 & 0 & 1 & \Delta t & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & \cdots \\
    0 & 0 & 0 & 1 & -\Delta t \frac{(u_{1_n} + u_{2_n}) \sin \theta_n}{m} & 0 & \Delta t \frac{\cos \theta_n}{m} & \Delta t \frac{\cos \theta_n}{m} & 0  & 0  & 0  & -1  & 0  & 0 & \cdots \\ 
    0 & 0 & 0 & 0 & 1 & \Delta t & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & \cdots \\ 
    0 & 0 & 0 & 0 & 0 & 1 & \Delta t \frac{L}{I} & -\Delta t \frac{L}{I} & 0 &  0 &  0 &  0 &  0 &  -1 & \cdots \\ 
    \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots
\end{bmatrix} 
$$

$$
\Delta y = \begin{bmatrix} \Delta X_0 \\ \Delta U_0 \\ \Delta X_1 \\ \Delta U_1 \\ \vdots \end{bmatrix}
\quad 
b = \begin{bmatrix} 
    X_{init} - X_0 \\ 
    p_{x_n} + \Delta t \cdot v_{x_n} -  p_{x_{n+1}} \\ 
    v_{x_n} - \Delta t \frac{(u_{1_n} + u_{2_n}) \sin \theta_n}{m} - v_{x_{n+1}} \\ 
    p_{y_n} + \Delta t \cdot v_{y_n} - p_{y_{n+1}} \\ 
    v_{y_n} + \Delta t \left(\frac{(u_{1_n} + u_{2_n}) \cos \theta_n}{m} - g \right) - v_{y_{n+1}} \\ 
    \theta_{n} + \Delta t \cdot \omega_n - \theta_{n+1} \\ 
    \omega_{n} + \Delta t \frac{L(u_{1_n} - u_{2_n})}{I} - \omega_{n+1} \\ 
    \vdots 
\end{bmatrix}
$$


Here, $X_{init}$ is the initial state of the quadrotor. As it is not defined in the constraints and can change for MPC, we must provide the functionality for it to be variable.

Based on this logic, we shall write a function to compute $A$ and $b$, as shown below.

In [17]:
def compute_A_b(x_init: np.ndarray, y: np.ndarray, N: int = 500) -> np.ndarray:
    y_i_dim = quadrotor.DIM_CONTROL + quadrotor.DIM_STATE
    # Convert inputs to NumPy arrays if they aren't already
    y = np.asarray(y, dtype=np.float64)
    x_init = np.asarray(x_init, dtype=np.float64)

    # Expected shapes based on N
    expected_y_shape = (y_i_dim * N,)
    expected_y_shape_alt = (y_i_dim * N, 1)

    expected_x_init_shape = (quadrotor.DIM_STATE,)
    expected_x_init_shape_alt = (quadrotor.DIM_STATE, 1)

    # Check the shape of x
    if y.shape not in [expected_y_shape, expected_y_shape_alt]:
        raise ValueError(
            f"Input y must be of shape {expected_y_shape} or {expected_y_shape_alt}, but got shape {y.shape}")

    if x_init.shape not in [expected_x_init_shape, expected_x_init_shape_alt]:
        raise ValueError(
            f"Input x_init must be of shape {expected_x_init_shape} or {expected_x_init_shape_alt}, but got shape {x_init.shape}")

    # Flatten if necessary
    if y.shape == expected_y_shape_alt:
        y = y.flatten()

    if x_init.shape == expected_x_init_shape_alt:
        x_init = x_init.flatten()

    num_constraints = 6 * N
    A = np.zeros((num_constraints, y_i_dim * N))
    b = np.zeros(num_constraints)

    A[:6, :] = np.eye(6, y_i_dim * N)
    b[:6] = x_init - y[:6]

    for n in range(0, N-1):
        i = n * y_i_dim  # Index for time step n
        idx_n = i
        idx_n1 = i + y_i_dim  # Index for time step n+1

        # Extract variables at time step n
        px = y[idx_n]
        vx = y[idx_n + 1]
        py = y[idx_n + 2]
        vy = y[idx_n + 3]
        theta = y[idx_n + 4]
        omega = y[idx_n + 5]
        u1 = y[idx_n + 6]
        u2 = y[idx_n + 7]

        # Extract variables at time step n+1
        px_next = y[idx_n1]
        vx_next = y[idx_n1 + 1]
        py_next = y[idx_n1 + 2]
        vy_next = y[idx_n1 + 3]
        theta_next = y[idx_n1 + 4]
        omega_next = y[idx_n1 + 5]

        # Row indices in A and b
        row = (n+1) * 6

        # First Constraint: px_{n+1} - px_n - Δt vx_n = 0
        A[row, idx_n] = -1        # Coefficient for px_n
        A[row, idx_n + 1] = -quadrotor.DT  # Coefficient for vx_n
        A[row, idx_n1] = 1        # Coefficient for px_{n+1}
        b[row] = 0

        # Second Constraint: vx_{n+1} - vx_n + Δt * [(u1 + u2) * sin(theta_n)/m] = 0
        A[row + 1, idx_n + 1] = -1  # Coefficient for vx_n
        A[row + 1, idx_n + 4] = quadrotor.DT * \
            ((u1 + u2) * np.cos(theta) / quadrotor.MASS)  # θ_n term
        A[row + 1, idx_n + 6] = quadrotor.DT * \
            (np.sin(theta) / quadrotor.MASS)  # u1_n term
        A[row + 1, idx_n + 7] = quadrotor.DT * \
            (np.sin(theta) / quadrotor.MASS)  # u2_n term
        A[row + 1, idx_n1 + 1] = 1  # Coefficient for vx_{n+1}
        b[row + 1] = 0

        # Repeat similar assignments for the remaining constraints...

        # Third Constraint: py_{n+1} - py_n - Δt vy_n = 0
        A[row + 2, idx_n + 2] = -1        # py_n
        A[row + 2, idx_n + 3] = -quadrotor.DT  # vy_n
        A[row + 2, idx_n1 + 2] = 1        # py_{n+1}
        b[row + 2] = 0

        # Fourth Constraint: vy_{n+1} - vy_n - Δt * [(u1 + u2) * cos(theta_n)/m - g] = 0
        A[row + 3, idx_n + 3] = -1  # vy_n
        A[row + 3, idx_n + 4] = quadrotor.DT * \
            ((u1 + u2) * np.sin(theta) / quadrotor.MASS)  # θ_n term
        A[row + 3, idx_n + 6] = -quadrotor.DT * \
            (np.cos(theta) / quadrotor.MASS)  # u1_n term
        A[row + 3, idx_n + 7] = -quadrotor.DT * \
            (np.cos(theta) / quadrotor.MASS)  # u2_n term
        A[row + 3, idx_n1 + 3] = 1  # vy_{n+1}
        b[row + 3] = quadrotor.DT * \
            quadrotor.GRAVITY_CONSTANT  # Include gravity term

        # Fifth Constraint: θ_{n+1} - θ_n - Δt ω_n = 0
        A[row + 4, idx_n + 4] = -1  # θ_n
        A[row + 4, idx_n + 5] = -quadrotor.DT  # ω_n
        A[row + 4, idx_n1 + 4] = 1  # θ_{n+1}
        b[row + 4] = 0

        # Sixth Constraint: ω_{n+1} - ω_n - Δt * [L(u1_n - u2_n)/I] = 0
        A[row + 5, idx_n + 5] = -1  # ω_n
        A[row + 5, idx_n + 6] = -quadrotor.DT * \
            (quadrotor.LENGTH / quadrotor.INERTIA)  # u1_n
        A[row + 5, idx_n + 7] = quadrotor.DT * \
            (quadrotor.LENGTH / quadrotor.INERTIA)   # u2_n
        A[row + 5, idx_n1 + 5] = 1  # ω_{n+1}
        b[row + 5] = 0

    return A, b


# compute_A_b(y=np.zeros(4000), x_init=np.zeros((6, 1)))

### 2. We would like the quadrotor to perfom a looping. Find and implement a suitable cost function to perform a looping and add constraint to maintain the thrust of each rotor between $0$ and $10$. Solve the problem using your own implementation of a SQP (levarging your code from Homework 2) with a large horizon to check that you can do a looping.

#### 2.1 Loop Definition and Discretisation
Firstly, let us define a loop. For the purpose of simplicity, we shall choose a simple circle having radius $3m$ and centered at $(0, 3)$. We shall write the parameteric equations of a circle based on this, 
$$\begin{align*}
    & x = h + r \sin \phi \Rightarrow x = 3 \sin \phi \\
    & y = k + r \cos \phi \Rightarrow y = 3 + 3 \cos \phi \\
    & \text{Where, } 0 \leq \phi \leq 2 \pi
\end{align*}$$

As this equation is continuous, it also needs to be discretised with respect to $\Delta t$. To do this, we must first choose a time horizon that is sufficient enough for the quadrotor is actually perform this loop. Initially, let us choose an arbitrary value of $N = 500$ iterations. The number of iterations multiplied by $\Delta t$ gives us the time horizon of $500 \times 0.04 = 20s$. Although this seems like a large amount of time, we can tune it later, upon observing the response of the drone. 

Based on this calculation, we can discretise our parameterised equations of a circle to be, 
$$\begin{align*}
    x_k &= 3 \sin (2 \pi k \Delta t) \\
    y_k &= 3 + 3 \cos \left(2\pi \Delta t \left(\frac{1}{2\Delta t} - k \right)\right) = 3  + 3 \cos\left(\pi \left(1 - 2 k \Delta t \right)\right)
\end{align*}$$
Here, $k$ represents the current iteration of the optimal control problem. Also note that the phase of $y_k$ has been shifted by $\frac{1}{2\Delta t}$ so that $(x_0, y_0) = (0, 0)$.

But this is only the 2D Cartesian Position of the drone. We also need to compute for the other elements of the state vector. Let's start of with $v_x$ and $v_y$. To do that, we can simply find the derivative of $x_k$ and $y_k$ with respect to $k$. This gives us, 

$$\begin{align*}
    {v_x}_k &= 6 \pi \Delta t \cos (2 \pi k \Delta t) \\
    {v_y}_k &= 6 \pi \Delta t \sin \left(\pi \left(1 - 2 k \Delta t \right)\right)
\end{align*}$$



#### 2.2 Cost Function Formulation 

The above equations essentially give us a trajectory that needs to be followed at any given time or iteration. Using this, we can easily formulate the cost to be, 
$$\frac{1}{2}\left(\sum_{k=0}^{N-1} (X_k - X_{k_des})^T Q (X_k - X_{k_des}) + U_n^T R U_n + (X_N - X_{N_des})^T Q (X_N - X_{N_des})\right)$$
Which penalizes deviation of the state from the desired state. This is equivalent to minimizing this cost (expanding and then ignoring constant terms, i.e. terms that do not depend on $X_n$)

$$\sum_{k=0}^{N-1} \left(\frac{1}{2}X_k^T Q X_k + \frac{1}{2}U_n^T R U_n - X_{k_des}^T Q X_k \right) + \frac{1}{2}X_k^T Q X_k + X_{N_des}^T Q X_N$$

Where, 
$$X_{k_des}^T = \begin{bmatrix}3 \sin (2 \pi k \Delta t) & 0 &  3 + 3cos \left(2\pi \Delta t \left(\frac{1}{2\Delta t} - k \right)\right)  & 0 & 0 & 0 \end{bmatrix}$$

The goal is to minimise this cost function, so we get, 

$$\min_{X_k, U_k}\frac{1}{2}\left(\sum_{k=0}^{N-1} X_k^T Q X_k + U_n^T R U_n - X_{k_des}^T Q X_k \right) + \frac{1}{2}X_k^T Q X_k + X_{N_des}^T Q X_N$$

Now, let us put it in the matrix form, 
$$\min_y \frac{1}{2} y^T P y + q^T y$$

Where, 
$$\begin{align}
    \notag &y = \begin{bmatrix} X_0 \\ U_0 \\ X_1 \\ U_1 \\ \vdots\end{bmatrix} && P = \begin{bmatrix} Q & 0 & 0 & 0 & \cdots\\ 0 & R & 0 & 0 & \cdots \\ 0 & 0 & Q & 0 & \cdots \\ 0 & 0 & 0 & R & \cdots \\ \vdots & \vdots & \vdots & \vdots & \ddots \end{bmatrix} &&& q^T = \begin{bmatrix} -X_{0_des}^T Q  & 0 & -X_{1_des}^T Q & 0 & \cdots\end{bmatrix}
\end{align}
$$

#### 2.3 Gradient of the cost function 

Let us recall that in lecture 2, a shortcut method was taught to find the gradient of a function. If a function is of the form, 
$\begin{align}
    \notag f(x) &= x^T P x + q^T x \\
    \notag \Rightarrow \nabla f(x) &= 2Px + q \\
    \notag \text{and} \quad \nabla ^2 f(x) &= 2 P
\end{align}$

Based on this logic, we shall write a function to compute the gradient of the cost, as shown below, 

In [26]:
def cost_grad(y: np.ndarray, Q_diag: np.ndarray, R_diag: np.ndarray, N: int = 500):
    y_i_dim = quadrotor.DIM_CONTROL + quadrotor.DIM_STATE
    # Convert inputs to NumPy arrays if they aren't already
    y = np.asarray(y, dtype=np.float64)
    Q_diag = np.asarray(Q_diag, dtype=np.float64)
    R_diag = np.asarray(R_diag, dtype=np.float64)

    # Expected shapes based on N
    expected_y_shape = (y_i_dim * N,)
    expected_y_shape_alt = (y_i_dim * N, 1)

    # Check the shape of x
    if y.shape not in [expected_y_shape, expected_y_shape_alt]:
        raise ValueError(
            f"Input y must be of shape {expected_y_shape} or {expected_y_shape_alt}, but got shape {y.shape}")

    if Q_diag.shape != (quadrotor.DIM_STATE, ):
        raise ValueError(
            f"Input Q_diag must be of shape {(quadrotor.DIM_STATE, )}, but got shape {Q_diag.shape}")

    if R_diag.shape != (quadrotor.DIM_CONTROL, ):
        raise ValueError(
            f"Input R_diag must be of shape {(quadrotor.DIM_CONTROL, )}, but got shape {R_diag.shape}")

    # Flatten if necessary
    if y.shape == expected_y_shape_alt:
        y = y.flatten()

    Q = np.diag(Q_diag)
    R = np.diag(R_diag)

    P = block_diag(*([block_diag(Q, R)] * N))
    q = np.zeros((quadrotor.DIM_STATE*N))
    for k in range(N):
        x_k = 3 * np.sin(2 * np.pi * k * quadrotor.DT)
        y_k = 3 + (3 * np.cos(2 * np.pi * quadrotor.DT *
                   ((1 / (2 * quadrotor.DT)) - k)))
        q[(k*quadrotor.DIM_STATE): ((k*quadrotor.DIM_STATE)+quadrotor.DIM_STATE)
          ] = -(np.array([[x_k], [0], [y_k], [0],  [0],  [0]]).T @ Q)
    q = q.reshape(-1, 1)
    return (P @ y) + q


cost_grad(y=np.zeros(4000), Q_diag=np.ones((6, )), R_diag=np.ones((2, )))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])


#### 2.X Optimal Control Problem Formulation
Now that we have a cost function, all we have to do is formulate the optimal control problem - 
$$\min_{X_k, U_k}\frac{1}{2}\left(\sum_{k=0}^{N-1} X_k^T Q X_k + U_n^T R U_n - X_{k_des}^T Q X_k \right) + \frac{1}{2}X_k^T Q X_k + X_{N_des}^T Q X_N\\
\text{Subject to}\quad X_{n+1} = X_n + \Delta t  f(X_n, U_n)\\ X_0 = X_{init}$$

#### 2.X+1 Including the Inequality Constraints 
It is given that the thrust of the rotors has to be between $0$ and $10$. Mathematically, 
$$ 0 \leq U_n \leq 10$$

Rewriting this in the matrix form of $G y \leq h$, we get, 
$$ 
\begin{bmatrix} 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1  \end{bmatrix}
\begin{bmatrix} p_x \\ v_x \\ p_y \\ v_y \\ \theta \\ \omega \\ u_1 \\ u_2 \end{bmatrix} = 
\begin{bmatrix} 10 \\ 0 \\ 10 \\ 0 \end{bmatrix}
$$

